In [104]:
import os
from pathlib import Path
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier

In [105]:
SPEC_PATH='/Users/zubrikhina/Documents/datasets/EEG_baseline_with_markers/cross_spectrum_short'

In [106]:
os.listdir(SPEC_PATH)

['MDD', '.DS_Store', 'Health']

In [107]:
def coherence_array(spectrum_file):
    diagnosis =0
    if spectrum_file.parent.stem=='MDD':
        diagnosis=1
    array =np.load(spectrum_file).flatten()
    array=np.append(array,diagnosis)
    return array

In [108]:
spectrum_raws=[]
for package in Path(SPEC_PATH).iterdir():
    if package.is_dir():
        for spectrum_file in package.iterdir():
            spectrum_raw =coherence_array(spectrum_file)
            spectrum_raws.append(spectrum_raw)

In [152]:
data =np.stack(spectrum_raws)

In [153]:
idx = np.argwhere(np.all(data[..., :] == 0, axis=0))
data = np.delete(data, idx, axis=1)

In [154]:
X=data[:, :-1]
Y= data[:, -1]

In [155]:
X.shape

(245, 420)

In [156]:
def build_models(X,y):    
    models = {
        'lr_l1': LogisticRegression(penalty='l1',solver='liblinear',C =1),
        'lr_l2': LogisticRegression(penalty='l2',solver='liblinear',C =1),
#         'rf_7': RandomForestClassifier(n_estimators=100, max_depth=7),
        'rf_5': RandomForestClassifier(n_estimators=100, max_depth=5),
#         'svm_stand_0.1': svm.SVC(C=0.1),
        'svm_stand_0.01': svm.SVC(C=0.01),
#         'knn': KNeighborsClassifier(n_neighbors=10),
#         'gb': GradientBoostingClassifier(max_depth=5),
        'mlp': MLPClassifier(hidden_layer_sizes=(70, 10), max_iter=500,alpha=1,learning_rate_init =0.001)
    }
    for name, model in models.items():
        strat_k_fold = StratifiedKFold(n_splits=10, shuffle=True)
        scores = cross_val_score(model, X, y, cv=strat_k_fold, scoring='accuracy')
        print(name, f"{scores.mean():.3f}+/-{scores.std():.3f}")
    

In [157]:
build_models(X,Y)

lr_l1 0.666+/-0.071
lr_l2 0.756+/-0.051
rf_5 0.878+/-0.091
svm_stand_0.01 0.621+/-0.015
mlp 0.645+/-0.032
